In [8]:
# Required Python Machine learning Packages
import pandas as pd
import numpy as np
import operator

# To split the dataset into train and test datasets
from sklearn.model_selection import train_test_split

# Classifiers
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, export_graphviz

from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.decomposition import PCA
from sklearn.ensemble import ExtraTreesClassifier

# To calculate the accuracy score of the model
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics

import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
def label_encode(df, columns):
    for col in columns:
        le = LabelEncoder()
        col_values_unique = list(df[col].unique())
        le_fitted = le.fit(col_values_unique)
 
        col_values = list(df[col].values)
        #le.classes_
        col_values_transformed = le.transform(col_values)
        df[col] = col_values_transformed

In [10]:
def classify(predictors, response, classifier = 'svm', kern='rbf', neighbors=3, kfolds=0, report=False, features_select=0):        
    # split X and y into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(predictors, response, test_size=0.20, random_state=1)

    
    if classifier == 'nb':
        cl = GaussianNB() # instantiate model
        msg = 'Naive Bayes'
    elif classifier == 'svm':
        cl = svm.SVC(kernel=kern) # instantiate model
        msg = 'SVM with ' + kern + ' kernel'
    elif classifier == 'knn':
        cl = KNeighborsRegressor(n_neighbors=neighbors)
        msg = 'KNN with k=' + str(neighbors)
    elif classifier == 'rf':
        # Instantiate model with  decision trees
        # Note that the number of decision trees is denoted
        # using the neighbors parameter, which is set to 3 by default
        cl = RandomForestRegressor(n_estimators = 1000, random_state = 42)
        msg = 'Random Forest with ' + str(neighbors) + ' decision trees'
    elif classifier == 'dtree':
        cl = DecisionTreeClassifier(min_samples_split=20, random_state=99)
        msg = 'Decision tree'
    elif classifier == 'logisticregression':
        cl = LogisticRegression()
        msg = 'Logistic Regression'
        
    model = cl.fit(X_train, y_train)
    y_pred_class = model.predict(X_test)
    print(msg + ' model accuracy score: ', metrics.accuracy_score(y_test, y_pred_class.round()))
    
    if kfolds > 0:
        # Perform k-fold cross validation
        scores = cross_val_score(model, predictors, response, cv=kfolds)
        print('Cross-validated score:', scores.mean())
    
    if report == True:
        print('Confusion matrix')
        print(metrics.confusion_matrix(y_test, y_pred_class)) # confusion matrix
        print('Classification report')
        target_names=['low','med','high']
        print(metrics.classification_report(y_test, y_pred_class, target_names=target_names))
        
        print('______________________________')
    
    if features_select > 0:
        test = SelectKBest(score_func=chi2, k=features_select)
        fit = test.fit(predictors, response)
        
        # summarize scores
        np.set_printoptions(precision=3)
        # print(fit.scores_)
        features_scores = {}
        for i in range(0, len(predictors.columns)):
            features_scores[predictors.columns[i]] = fit.scores_[i]
            
        # print(features_scores)
        sorted_features_scores = sorted(features_scores.items(), key=operator.itemgetter(1))
        print(sorted_features_scores[-5:])
        
    
        
    
        

In [11]:
def decompose(predictors):
    # feature extraction
    pca = PCA(n_components=3)
    fit = pca.fit(predictors)
    
    # summarize components
    # print(fit.explained_variance_ratio_)
    # print(fit.components_)
    return pd.DataFrame(fit.components_)

In [12]:
# df = pd.read_csv("master_no_missing_vals_1532369164.csv")
# df = pd.read_csv("master_binary_outcomes_v1_07_27_2018.csv")
df = pd.read_csv("pima_indians_diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [13]:
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [14]:

# define X and y
X = df[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age']]
label_encode(X, X.columns.values)


y = df[['Outcome']]
label_encode(y, y.columns.values)
y = y['Outcome']




/Users/dmitriyb/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
#classify(X, y, classifier='nb', kfolds=10, report=True, features_select=3)
#classify(X, y, classifier='svm', kern='rbf', kfolds=10, report=False, features_select=3)
#X_pca = decompose(X)
#classify(X_pca, y, classifier='svm', kern='rbf', kfolds=10, report=False, features_select=3)
#print(X_pca.shape)
# classify(X, y, classifier='svm', kern='linear', kfolds=10, report=True)
# classify(X, y, classifier='svm', kern='poly', kfolds=10, report=True)
# classify(X, y, classifier='knn', neighbors=3, kfolds=10)
#classify(X, y, classifier='knn', neighbors=5, kfolds=10)
# classify(X, y, classifier='knn', neighbors=7)
classify(X, y, classifier='rf', neighbors=1000, kfolds=10)
# classify(X, y, classifier='rf', neighbors=2000)
# classify(X, y, classifier='dtree', kfolds=10)
#classify(X, y, classifier='logisticregression', kfolds=10)